In [2]:
cd "./same_class/car2car/"

/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/latent/pytorch-pretrained-BigGAN/other_class/same_class


In [3]:
%load_ext autoreload
%autoreload 2

# target image
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal, one_hot_from_int)
import logging


logging.basicConfig(level=logging.INFO)

model = BigGAN.from_pretrained('biggan-deep-128')

truncation = 0.5
class_vector = one_hot_from_int(581)
#class_vector = one_hot_from_names(["car"], batch_size=1)
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=1, seed = 5)

noise_vector = torch.from_numpy(noise_vector)
class_vector = torch.from_numpy(class_vector)


with torch.no_grad():
    output = model(noise_vector, class_vector, truncation)

save_as_images(output, "car_target")


INFO:pytorch_pretrained_biggan.model:loading model biggan-deep-128 from cache at /Users/xuejiaxin/.pytorch_pretrained_biggan/6371c3777477e4e75187da1b9b526561aac3134f38c7299a3438009ae560e20d.3434ebdfa74a8c17e0e56061cfd905fa163c92f110e88df77b47da6ce9910b48
INFO:pytorch_pretrained_biggan.model:Model config {
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 1000,
  "output_dim": 128,
  "z_dim": 128
}

INFO:pytorch_pretrained_biggan.utils:Saving image to car_target_

In [4]:
from torchvision.models import squeezenet1_0
from tqdm import trange

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = model.eval().to(DEVICE)


semantic_model = squeezenet1_0(pretrained=True).to(DEVICE)
semantic_model.classifier = torch.nn.Sequential(
    torch.nn.Flatten()
    )
semantic_model = semantic_model.eval()

trunction = 0.5

class_vector = one_hot_from_int(c)
#class_vector = one_hot_from_names(['car'], batch_size=1)
class_vector = torch.from_numpy(class_vector)

noise = truncated_noise_sample(truncation=truncation, batch_size=1, seed=99)
noise = torch.nn.Parameter(torch.tensor(noise, requires_grad=True).float().to(DEVICE))
noise_optim = torch.optim.Adam([noise], lr=0.05)



L = []
L_pixel = []
L_semantic = []

for iteration in trange(0, 200):
    noise_optim.zero_grad()

    y_hat = model(noise, class_vector, truncation)

    semantic_loss = ((semantic_model(y_hat) - semantic_model(output)) ** 2).mean() ** .5 #-cos_sim(semantic_model(y_hat), semantic_model(output))
    L_semantic.append(semantic_loss.item())

    pixel_loss = abs(y_hat - output).mean()
    L_pixel.append(pixel_loss.item())

    loss = semantic_loss + 30 * pixel_loss
    L.append(loss.item())

    loss.backward()
    noise_optim.step()

    if iteration % 2 == 0:
        save_as_images(y_hat, f"car2car_{iteration}")

100%|██████████| 200/200 [17:19<00:00,  5.20s/it]


In [1]:
import matplotlib.pyplot as plt


plt.plot(L)
plt.plot([x*30 for x in L_pixel], 'r')
plt.plot(L_semantic, 'b')

plt.show()

NameError: name 'L' is not defined